### Object Detection
![](https://cn.bing.com/th?id=OIP.wEsP8kpHxFkqHkUxSOLQ-QHaDD&pid=Api&rs=1&p=0)  
![](https://image.slidesharecdn.com/cs231n2017lecture11-171030090644/95/cs231n-2017-lecture11-detection-and-segmentation-49-638.jpg?cb=1509354489)

### 1 stage detector
- classification and localization from the same network simultaneously
![](https://cn.bing.com/th?id=OIP.0ogh_9xVCQ2AMbW6ehC9YQHaEc&pid=Api&rs=1&p=0)

### Faster R-CNN Architecture
![](https://www.researchgate.net/profile/Zhipeng_Deng/publication/324903264/figure/download/fig2/AS:640145124499471@1529633899620/The-architecture-of-Faster-R-CNN.png?_sg=ifJyIJAwbp6N2Qnshu5HUMS8HrxLIoUn2RI4wXOJYrJBO96_th2NbQJO9aaMdVCMhSRUs9np2qI)  

![](https://images2015.cnblogs.com/blog/1058268/201612/1058268-20161221111645057-64418700.jpg)


### YOLO
![](https://images2015.cnblogs.com/blog/1058268/201612/1058268-20161221112032729-1756031476.jpg)  

### Single Shot multibox Detector
![](https://image.slidesharecdn.com/objectdetectors1-170506043442/95/deep-object-detectors-1-20166-68-638.jpg?cb=1494207275)

![](http://www.deeplearningessentials.science/img/singleShotDetector/SSDModel.png)

- Base Network(VGG 16)
- FC6 and FC7 converted to convolution layer
- Auxiliary structure to produce detections
- collections of BBoxes and Scores

- we draw the Conv4_3 to be 8 × 8 spatially (it should be 38 × 38). For each cell (also called location), it makes 4 object predictions.
![](https://cdn-images-1.medium.com/max/800/1*vNaiiFUVwCfzx1znKiFYYw.jpeg)

- Conv4_3 makes a total of __38 × 38 × 4 predictions(four bounding boxes as above right figure)__: four predictions per cell regardless of the depth of the feature maps. As expected, many predictions contain no object. SSD reserves a class “0” to indicate it has no objects.

![](https://cdn-images-1.medium.com/max/1200/1*mvF9g_zH2DaQK2KgutndFg.jpeg)

- Apply a 3x3 convolution filter to make a prediction for the location and the class.  
![](https://cdn-images-1.medium.com/max/800/1*mGtRvk9g1PVWw6IkjEryuA.jpeg)  

- For example, in Conv4_3, we apply four 3 × 3 filters to map 512 input channels to 25 output channels.
![](https://cdn-images-1.medium.com/max/800/1*fxy_hGpFB8V9ZA3KYTGgfg.jpeg)

#### Multi-scale feature maps for detection
![](https://cdn-images-1.medium.com/max/800/1*ZOeQglB3OQ3Qrc_L3r83dg.jpeg)  
- Lower resolution feature maps (right) detects larger scale objects.

![](https://cdn-images-1.medium.com/max/1200/1*up-gIJ9rPkHXUGRoqWuULQ.jpeg)

#### default boundary box
- Conceptually, the ground truth boundary boxes can be partitioned into clusters with each cluster represented by a default boundary box (the centroid of the cluster). So, instead of making random guesses, we can start the guesses based on those default boxes.

- SSD also keeps the default boxes to a minimum (4 or 6) with one prediction per default box
- the boundary box predictions are relative to the default boundary boxes at each cell (∆cx, ∆cy, w, h), i.e. the offsets to the default box at each cell.
- For each feature map layers, it shares the same set of default boxes centered at the corresponding cell. But different layers use different sets of default boxes to customize object detections at different resolutions. The 4 green boxes below illustrates 4 default boundary boxes.

#### Choosing the default boundary boxes
- __Default boundary boxes are chosen manually. SSD defines a scale value for each feature map layer. Starting from the left, Conv4_3 detects objects at the smallest scale 0.2 (or 0.1 sometimes) and then increases linearly to the rightmost layer at a scale of 0.9.__

![](https://cdn-images-1.medium.com/max/800/1*JEakOqmXwsYs0heVDmAuGA.png)

#### Matching strategy

- SSD predictions are classified as positive matches or negative matches. SSD only uses positive matches in calculating the localization cost (the mismatch of the boundary box). If the corresponding default boundary box (not the predicted boundary box) has an IoU greater than 0.5 with the ground truth, the match is positive. Otherwise, it is negative. (IoU, the intersection over the union, is the ratio between the intersected area over the joined area for two regions.)

![](https://cdn-images-1.medium.com/max/800/1*gpxMpC_-HOZAYDYISkp2MA.jpeg)

#### Multi-scale feature maps & default boundary boxes
![](https://cdn-images-1.medium.com/max/800/1*-KVIXjvBO5m2MQZrzWx-wg.png)

![](https://cdn-images-1.medium.com/max/800/1*sZUWR2XgCAJ6AXM5NXYjNg.png)

### Loss Function
![](https://cdn-images-1.medium.com/max/800/1*cIE7bbicMOokWQ6w41I-NA.png)

####  confidence loss
![](https://cdn-images-1.medium.com/max/800/1*OTVm8L9RoAKtwl3XEQNkzA.png)


- The final loss function is computes as:
![](https://cdn-images-1.medium.com/max/800/1*I79wnE3675HYqcMKMm7yHg.png)

### non-maximum suppression (nms)

- SSD uses non-maximum suppression to remove duplicate predictions pointing to the same object. SSD sorts the predictions by the confidence scores. Start from the top confidence prediction, SSD evaluates whether any previously predicted boundary boxes have an IoU higher than 0.45 with the current prediction for the same class. If found, the current prediction will be ignored. At most, we keep the top 200 predictions per image.

![](https://cdn-images-1.medium.com/max/800/1*aCMSCIUVkHrQI7QQEVnxew.png)

#### Ref

[A port of SSD: Single Shot MultiBox Detector to Keras framework.](https://github.com/rykov8/ssd_keras)  


[SSD blog](https://medium.com/@jonathan_hui/ssd-object-detection-single-shot-multibox-detector-for-real-time-processing-9bd8deac0e06)

# - Additional Explanation

### Default Boxes and Aspect Ratios
![](https://image.slidesharecdn.com/pr12ssd-190106144735/95/pr132-ssd-single-shot-multibox-detector-11-638.jpg?cb=1546786179)

- 6*(21+4) : 6 menas 6 bounding boxes at each location
- total size of each feature map(m X n): (c(21) + 4(cx,cy,w,h))*6(num of  bounding boxes) *m * n